In [12]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.9/221.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [14]:
messages_try = [
    {
        "role": "user",
        "content": "I want you to pick a crypto currency, such as Bitcoin, Ethereum, etc. Then tell me how its price changed in the stated one hour time interval (I want you to make it up, either increase, steady, or decrease)",
    },
    {
        "role": "assistant",
        "content": "Understood! What else?",
    },
    {
        "role": "user",
        "content": "Also create 10 tweets about that crypto currency. Probably those tweets will explain the possible reasons for its price change. At the end, provide a summary of what happened to the price and for what reason!"
    },
    {
        "role": "assistant",
        "content": "Could you give me one example about how my output should look like?",
    },
    {
        "role": "user",
        "content": """Price movement: Bitcoin price increased sharply in the given one-hour time interval, increased %10, from $30k to $33k. Tweets: 1) BlackRock's Bitcoin ETF application could reshape crypto investments. #BlackRock #BitcoinETF.
2) With BlackRock's move, are we witnessing a new era for Bitcoin in mainstream finance? #CryptoEvolution
3) The Bitcoin community is abuzz with BlackRock's ETF application, signaling mainstream acceptance. #BitcoinBuzz
4) BlackRock's entry into Bitcoin with an ETF might be a game-changer for institutional investors. #CryptoShift
5) BlackRock and Bitcoin, a powerful combination that could accelerate crypto adoption. #BlackRockBitcoin
6) Exciting times for Bitcoin with its latest security and scalability upgrade. #BitcoinTech
7) Bitcoin's journey to becoming a legal tender globally marks a new financial era. #BitcoinGlobal
8) Renewable energy is powering Bitcoin mining, leading to a greener crypto future. #GreenBitcoin
9) Universities are now teaching Bitcoin and blockchain, marking a shift in finance education. #BitcoinLearning
10) Bitcoin is becoming a tool for financial freedom worldwide, a true digital revolution. #BitcoinFreedom.
Summary:
Bitcoin's price jump from $30k to $33k was driven by BlackRock's Bitcoin ETF application, boosting mainstream acceptance and investor confidence, along with technological advancements in the cryptocurrency.
"""
    },
    {
        "role": "user",
        "content": "Please provide a random output in the form of example given above!",
    }
]

In [21]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key = "932771f9c30548d2a849192433ef3ac8",
    api_version = "2023-10-01-preview",
    azure_endpoint = "https://synopticgpt4.openai.azure.com"
    )

deployment_name="gpt4" #This will correspond to the custom name you chose for your deployment when you deployed a model.

# Send a completion call to generate an answer
print('Sending a test completion job')
#start_phrase = 'Write a tagline for an ice cream shop. '
response = client.chat.completions.create(model=deployment_name, messages=messages_try, max_tokens=500, n=125)
print(response.choices[0])

Sending a test completion job
Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Price movement: In the given one-hour interval, the Ethereum price observed a steady state of around $2000.\n\nTweets:\n1) Amazon stepping into crypto space? Rumors hinting Ethereum might be their first choice! #AmazonCrypto #Ethereum \n2) Interesting developments to watch today in the Ethereum community. Is the Amazon partnership real? #EthereumNews \n3) Ethereum maintained its stable position today despite some market fluctuations. #EthereumSteady \n4) Given the recent Ethereum steady market, traders are watching the potential Amazon collaboration closely. #EthereumAmazon\n5) Ethereum's scalability upgrade is stalled, Having an impact on its stable market position. #EthereumUpgrade\n6) Could Amazon's much-awaited entry into cryptocurrency be the turning point for Ethereum? #AmazonEntry #Crypto\n7) Even without the Amazon news, Ethereum's intrinsic qualities make it a fascinating

In [17]:
import re

In [22]:
#list_of_dicts = []
for i in range(125):
  data_string = response.choices[i].message.content
  # Splitting the data into sections
  sections = re.split(r'(Price movement:|Tweets:|Summary:)', data_string)

  # Cleaning and organizing the data
  data_dict = {}
  for i in range(1, len(sections), 2):
      key = sections[i].strip()
      value = sections[i + 1].strip()
      data_dict[key] = value

  # Output the dictionary
  list_of_dicts.append(data_dict)

In [23]:
data_dicts = list_of_dicts

In [34]:
len(data_dicts)

250

In [35]:
import csv

# Define the CSV file name
csv_file = "raw_data.csv"

# Write to CSV
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    # Assuming all dictionaries have the same keys, use the keys from the first dictionary as fieldnames
    fieldnames = data_dicts[0].keys()
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    # Write the header
    writer.writeheader()

    # Write the data rows
    for data in data_dicts:
        writer.writerow(data)

print(f"Data written to {csv_file}")

Data written to raw_data.csv


In [1]:
import pandas as pd

In [ ]:
import pandas as pd

file_path = '/content/raw_data.csv'

dataframe = pd.read_csv(file_path)

print(dataframe)

In [38]:
# Concatenate the first two columns and append "concat well"
dataframe['Text'] = "###Human:\nCould you explain why " + dataframe["Price movement:"] + "\n\n###Input:\n" + dataframe['Tweets:'] + "\n\n###Assistant:\n" + dataframe['Summary:']

In [39]:
train_df = dataframe[['Text']]

In [ ]:
train_df

In [41]:
file_path = '/content/train_data.csv'

# Save the DataFrame to a CSV file
train_df.to_csv(file_path, index=False)